#### Import Abstracts

In [1]:
import pandas as pd
data = pd.read_csv("StanfordCo.csv")
data = data.drop('Unnamed: 0', axis=1)
data

,S,P,O,label,dboSource,Abstracts,Triples
0,http://yago-knowledge.org/resource/Van_Helsing...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Van Helsing (film),Van_Helsing_(film),Van Helsing is a 2004 action gothic horror fil...,"[['is', '2004 action gothic horror film writte..."
1,http://yago-knowledge.org/resource/One_Flew_Ov...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,One Flew Over the Cuckoo's Nest (film),One_Flew_Over_the_Cuckoo's_Nest_(film),One Flew Over the Cuckoo's Nest is a 1975 Amer...,"[['is', '1975 American psychological comedy dr..."
2,http://yago-knowledge.org/resource/The_WB,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,The WB,The_WB,"The WB Television Network (for Warner Bros., o...","[['was', 'television network launched on broad..."
3,http://yago-knowledge.org/resource/This_Must_B...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,This Must Be the Place (film),This_Must_Be_the_Place_(film),This Must Be the Place is a 2011 Italian-Frenc...,"[['is directed', 'Sorrentino'], ['is Irish', '..."
4,"http://yago-knowledge.org/resource/Monsters,_Inc.",http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Monsters, Inc.","Monsters,_Inc.","Monsters, Inc. (also known as Monsters, Incorp...","[['is', '2001 animated monster comedy film for..."
...,...,...,...,...,...,...,...
12758,http://yago-knowledge.org/resource/Lincoln_Col...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,"Lincoln College, Oxford","Lincoln_College,_Oxford","Lincoln College (formally, The College of the ...","[['was founded by', 'then Bishop'], ['was foun..."
12759,http://yago-knowledge.org/resource/104th_Unite...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,104th United States Congress,104th_United_States_Congress,The 104th United States Congress was a meeting...,"[['was', 'meeting'], ['was', 'meeting of branc..."
12760,http://yago-knowledge.org/resource/London_Busi...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,London Business School,London_Business_School,London Business School (LBS) is a business sch...,"[['college of', 'federal University'], ['colle..."
12761,http://yago-knowledge.org/resource/Political_s...,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,popularEntity,Political science,Political_science,Political science is the scientific study of p...,"[['is scientific study of', 'politics'], ['is ..."


In [2]:
data['Triples'][0]

"[['is', '2004 action gothic horror film written'], ['is', '2004 action horror film written by Stephen Sommers'], ['is', 'gothic written'], ['is', '2004 action horror film'], ['is', '2004 action gothic horror film'], ['is', '2004 action horror film written'], ['is', '2004 action gothic horror film written by Stephen Sommers'], ['stars Hugh Jackman as', 'monster hunter Van Helsing'], ['stars Hugh Jackman as', 'Dutch monster hunter Van Helsing'], ['stars', 'Hugh Jackman'], ['is homage to', 'Universal Horror Monster films from 1930s'], ['is', 'homage produced by Universal Pictures'], ['is', 'homage to Universal Horror Monster films produced'], ['is', 'homage also produced by Universal Pictures'], ['is', 'homage to Universal Horror Monster films from 1930s also produced by Universal Pictures'], ['is', 'homage'], ['is', 'homage to Universal Horror Monster films from 1930s produced by Universal Pictures'], ['is', 'homage to Universal Horror Monster films from 1930s also produced'], ['is', 'h

#### Model

##### Whole code (Too slow to run)

In [ ]:
from openie import StanfordOpenIE
from transformers import pipeline
import spacy

properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

unmasker = pipeline('fill-mask', model='bert-base-uncased')
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0
with StanfordOpenIE(properties=properties) as client:
    
    for s,p,o,label,dbo,abstract in data.values:

        if type(abstract) is float:
            datatrips.append([])
            print(j,label)
            j += 1
            continue

        print(j,label)


        #text = abstract
        #print('Text: %s.' % text)
        for triple in client.annotate(abstract):
            #print('|-', triple)
            triples.append(triple)

        for triple in triples:
            # turn triple into string 
            name = triples[0]['subject']
            string = (triple['subject'],triple['relation'],triple['object'])
            string = (' '.join(string))
            tokens = nlp(string)

            # if something in chunk
            if len(chunks) > 0:

                # if subject related to entity
                if any(good['subject'].casefold() == item for item in item_list) or good['subject'] in name or good['subject'] in label:
                    # if chunk is 'is' or 'was'
                    if chunks[-1] == 'is' or chunks[-1] == 'was':
                        ob = nlp(good['object'])

                        # get noun chunks from object
                        for b in ob.noun_chunks:
                            # make triple
                            slm = ("%s %s [MASK] %s" %(label, chunks[-1], b.text))
                            # check not already in triples
                            if ('%s - %s - %s' %(name, unmasker(slm)[0]['token_str'], b.text)) not in trips:
                                trips.append('%s - %s - %s' %(name, unmasker(slm)[0]['token_str'], b.text))

                    else:
                        ob = nlp(good['object'])

                        # get noun chunks from object
                        for b in ob.noun_chunks:
                            # mkae triple
                            slm = ("%s [MASK] %s %s" %(label, chunks[-1], b.text))
                            # if MASK is punctuation, remove MASK
                            if unmasker(slm)[0]['token_str'] in punc:
                                if ('%s - %s - %s' %(name, chunks[-1], b.text)) not in trips:
                                    # check not already in triples
                                    trips.append('%s - %s - %s' %(name, chunks[-1], b.text))

                            else:
                                if ('%s - %s%s - %s' %(name, unmasker(slm)[0]['token_str'], chunks[-1].capitalize(), b.text)) not in trips:
                                    # check not already in triples
                                    trips.append('%s - %s%s - %s' %(name, unmasker(slm)[0]['token_str'], chunks[-1].capitalize(), b.text))

                                    
        else:
            trips.append()
                # empty chunk list
                chunks = []

            # get chunk head 
            for chunk in tokens.noun_chunks:
                chunks.append(chunk.root.head.text)

            #print(triple)
            good = triple

        #for trip in trips:
            #print(trip)

        datatrips.append(trips)
        triples = []
        trips = []
        j += 1

##### Extract triples (Around 3 hours)

In [4]:
from openie import StanfordOpenIE
from transformers import pipeline
import spacy

properties = {
    'openie.affinity_probability_cap': 1 / 3,
}

unmasker = pipeline('fill-mask', model='bert-base-uncased')
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrip = []
j = 0
first_name = 0
with StanfordOpenIE(properties=properties) as client:
    
    for s,p,o,label,dbo,abstract in data.values:

        if type(abstract) is float:
            datatrip.append([])
            print(j,label, 'nothing')
            j += 1
            continue

        print(j,label)


        #text = abstract
        #print('Text: %s.' % text)
        for triple in client.annotate(abstract):
            if first_name == 0:
                name = triple['subject']
                first_name += 1
            #print('|-', triple)
            if any(triple['subject'].casefold() == item for item in item_list) or triple['subject'] in name or triple['subject'] in label:
                triples.append(triple)
        first_name = 0
            
        datatrip.append(triples)
        triples = []
        j += 1

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 Van Helsing (film)
Starting server with command: java -Xmx8G -cp /Users/ab/.stanfordnlp_resources/stanford-corenlp-4.5.3/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-c349f68d4a8045e9.props -preload openie
1 One Flew Over the Cuckoo's Nest (film)
2 The WB
3 This Must Be the Place (film)
4 Monsters, Inc.
5 Paul Thomas Anderson
6 Tim Van Patten
7 Silkeborg IF
8 Bram Stoker
9 Trier
10 Emma Roberts
11 Peter Dinklage
12 Christopher Boyes
13 Joe Jackson (musician)
14 Astralwerks
15 MCA Inc.
16 Ja Rule
17 Grammy Award for Best Male Rock Vocal Performance
18 Liechtenstein national football team
19 Rich Man, Poor Man (miniseries)
20 Monster House (film)
21 Windsor County, Vermont
22 Basingstoke
23 John Stephenson (actor)
24 Arnold Schwarzenegger
25 Carnegie Mellon University
26 Babel (film)
27 City of London
28 RMIT University
29 Satellite Award for Best Original Song
30 Mali
31 Cowpunk

In [ ]:
data['Triples'] = datatrip
#data.to_csv('Stanfordpart1.csv')
data

In [19]:
testing = []
done = []
j = 0
for s,p,o,label,dbo,abstracts,triples in data.values:
    print(j, label)
    if triples is None:
        done.append([])
    for triple in triples:
        testing.append([triple['relation'], triple['object']])
    
    done.append(testing)
    testing = []
    j += 1

In [23]:
data1 = data
data1.Triples = done
data1.to_csv('StanfordCo.csv')

In [27]:
data['Triples'][0][0][0]

##### Process Triples (Too long to run)

In [2]:
import ast
from transformers import pipeline
import spacy

unmasker = pipeline('fill-mask', model='bert-base-uncased', top_k=1, framework='pt', device = -1)
chunks = []
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
item_list = ['he', 'his', 'it', 'she', 'her', 'they', 'them']
trips = []
triples = []
nlp = spacy.load("en_core_web_sm")
datatrips = []
j = 0
trips = []
for s,p,o,label,dbo,abstract,triples in data.values:
    
    if type(abstract) is float:
            datatrips.append([])
            print(j,label, 'nothing')
            j += 1
            continue
            
    print(j,label)
    
    for triple in ast.literal_eval(triples):
        # turn triple into string 
        string = (label,triple[0],triple[1])
        string = (' '.join(string))
        tokens = nlp(string)

        for chunk in tokens.noun_chunks:
            chunks.append(chunk.root.head.text)

        # if something in chunk
        if len(chunks) > 0:

            if chunks[-1] == 'is' or chunks[-1] == 'was':
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    slm = ("%s %s [MASK] %s" %(label, chunks[-1], b.text))
                    # check not already in triples
                    if ([unmasker(slm)[0]['token_str'], b.text]) not in trips:
                        trips.append([unmasker(slm)[0]['token_str'], b.text])

            else:
                ob = nlp(triple[1])

                # get noun chunks from object
                for b in ob.noun_chunks:
                    # make triple
                    slm = ("%s [MASK] %s %s" %(label, chunks[-1], b.text))
                    # if MASK is punctuation, remove MASK
                    if unmasker(slm)[0]['token_str'] in punc:
                        if ([chunks[-1], b.text]) not in trips:
                            # check not already in triples
                            trips.append([chunks[-1], b.text])

                    else:
                        if (["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text]) not in trips:
                            # check not already in triples
                            trips.append(["%s%s" %(unmasker(slm)[0]['token_str'], chunks[-1].capitalize()),b.text])

        else:
            trips.append(['%s' %(triple[0]),'%s' %(triple[1])])
            # empty chunk list
        chunks = []

    datatrips.append(trips)
    trips = []
    j += 1

/Users/ab/opt/anaconda3/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 Van Helsing (film)
1 One Flew Over the Cuckoo's Nest (film)
2 The WB
3 This Must Be the Place (film)
4 Monsters, Inc.
5 Paul Thomas Anderson
6 Tim Van Patten
7 Silkeborg IF
8 Bram Stoker
9 Trier
10 Emma Roberts
11 Peter Dinklage
12 Christopher Boyes
13 Joe Jackson (musician)
14 Astralwerks
15 MCA Inc.
16 Ja Rule
17 Grammy Award for Best Male Rock Vocal Performance
18 Liechtenstein national football team
19 Rich Man, Poor Man (miniseries)
20 Monster House (film)
21 Windsor County, Vermont
22 Basingstoke
23 John Stephenson (actor)
24 Arnold Schwarzenegger
25 Carnegie Mellon University
26 Babel (film)
27 City of London
28 RMIT University
29 Satellite Award for Best Original Song
30 Mali
31 Cowpunk
32 Porter County, Indiana
33 Armando Iannucci
34 Orduspor
35 South Somerset
36 Alkmaar
37 Mick Jagger
38 F.C. Hansa Rostock nothing
39 Granada CF
40 London Borough of Camden
41 U.S. Sassuolo Calcio
42 Dubai
43 University of Queensland
44 Michael Shamberg
45 Hugo Award for Best Professional Art

In [6]:
data['StanfordFinaltrips'] = datatrips
data.to_csv('StanfordFinalTrips.csv')

In [1]:
final_trips = []
import ast
trips = []
for s,p,o,label,dbo,abstract,_,triples in data.values:
    if type(abstract) == float:
        final_trips.append([])
        continue
    
    for triple in triples:
        if triple[0] == 'is' or triple[0] == 'also' or triple[0] == 'an' or triple[0] == 'a' or triple[0] == 'was' or triple[0] == 'as':
            simpletok = triple[1].split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(triple[0],'- type'," ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append(['type'," ".join(simpletok[1:])])
            else:
                print(triple[0], '-', 'type', triple[1], '-', label)
                trips.append(['type', triple[1]])
        else:
            simpletok = triple[1].split(' ')
            if simpletok[0] == 'the' or simpletok[0] == 'a' or simpletok[0] == 'an' or simpletok[0] == 'he' or simpletok[0] == 'her' or simpletok[0] == 'it' or simpletok[0] == triple[0] or simpletok[0] == 'also':
                print(" ".join(simpletok[1:]), '-', label)
                if len(" ".join(simpletok[1:])) == 0:
                       print(label, triple[0], '- Removed' )
                else:
                       trips.append([triple[0], " ".join(simpletok[1:])])
            else:
                trips.append(triple)
    final_trips.append(trips)
    trips = []

NameError: name 'data' is not defined

In [29]:
pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.7 MB/s eta 0:00:00ta 0:00:01
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
Note: you may need to restart the kernel to use updated packages.


In [30]:
trips = []
done = []
nouns = []
import spacy 
import nltk
nlp = spacy.load("en_core_web_sm")

for triples in final_trips: 
    for triple in triples:
        dd = triple[0].split(' ')
        if len(dd) > 1:
      #print(triple[0], '-', label, triple[0].split(' ')[1], triple[1])
            tokens = nlp(triple[1])
            for token in tokens.noun_chunks:
                nouns.append(token)
            if len(nouns) == 1:
                print(triple[0], '-', label, triple[0].split(' ')[1], nouns[0])
                trips.append([triple[0].split(' ')[1], nouns[0]])
            else:
                tokenized = nltk.word_tokenize(triple[1])
                pos = nltk.pos_tag(tokenized)
                for i in pos:
                    if i[1] == 'CD' or i[1] == 'NN':
                        print(triple[0], '-', label, triple[0].split(' ')[1], i[0])
                        trips.append([triple[0].split(' ')[1], i[0]])
            nouns = []
        else:
            trips.append(triple)
    done.append(trips)
    trips = []

has participated at_time - Martin Sheen participated times
has participated at_time - Martin Sheen participated several times
Moving in - Martin Sheen in 2005
released at_time - Martin Sheen at_time earlier year
released at_time - Martin Sheen at_time same year
released at_time - Martin Sheen at_time year
released at_time - Martin Sheen at_time earlier same year
Together formed - Martin Sheen formed two
Together formed - Martin Sheen formed man
Together formed - Martin Sheen formed band
Together formed - Martin Sheen formed two
Together formed - Martin Sheen formed man
Together formed - Martin Sheen formed band
's known for - Martin Sheen known 2015
co-founded in - Martin Sheen in 1998
Economically is ranked third - Martin Sheen is third area
Economically is ranked third - Martin Sheen is third metropolitan area
Economically is ranked third - Martin Sheen is area
Economically is ranked - Martin Sheen is third area
become worldwide - Martin Sheen worldwide grossing film
Declared in - Ma

In [31]:
trips = []
final = []
for triples in done:
    for triple in triples:
        obj = str(triple[1]).split(" ")
        obj = "_".join(obj)
        trips.append([triple[0], obj])
    final.append(trips)
    trips = []
len(final), final[:5]

(12763,
 [[['type', '2004_action_gothic_horror_film'],
   ['inspiredBy', '2004_action_horror_film'],
   ['directedBy', 'Stephen_Sommers'],
   ['vanHelsing', 'gothic'],
   ['type', '2004_action_horror_film'],
   ['inspiredBy', '2004_action_gothic_horror_film'],
   ['as', 'monster_hunter_Van_Helsing'],
   ['as', 'Dutch_monster_hunter_Van_Helsing'],
   ['stars', 'Hugh_Jackman'],
   ['accordingTo', 'Universal_Horror_Monster_films'],
   ['inspiredBy', 'homage'],
   ['producedBy', 'Universal_Pictures'],
   ['to', 'homage'],
   ['producedBy', 'Universal_Horror_Monster_films'],
   ['builtBy', '1930s'],
   ['type', 'homage'],
   ['from', 'homage'],
   ['adaptedFrom', 'Universal_Horror_Monster_films'],
   ['from', '1930s'],
   ['produced', 'homage'],
   ['vanHelsing', 'homage']],
  [['directed', '1975_American_psychological_comedy_drama_film'],
   ['inspiredBy', '1975_comedy_drama_film'],
   ['directedBy', 'Miloš_Forman'],
   ['flewOver', 'Nest'],
   ['type', '1975_comedy_drama_film'],
   ['type

In [32]:
data['StanfordFinalProTrips'] = final
data.to_csv('StanfordFinalProTrips.csv')

In [33]:
pip install rfc3987

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


In [40]:
import rdflib
from rdflib import Graph
from rdflib.namespace import RDF
from rdflib import Literal, XSD
from rfc3987 import parse
flag = True
g = Graph()
k = 0
n = 0
for s,p,o,label,dbo,abstract,_,_,triples in data.values:
  #print(k,label)
    if type(abstract) == float:
        #print('nothing')
        k+=1
        continue
    for triple in triples:
        flag = True
        try:
            int(triple[1])
        except:
            flag = False 
    
        if flag is True:
            rel = triple[0].replace("'", '')
            rel = rel.replace('"', '')
            rel = rel.replace('`', '')
            rel = rel.replace('.', '')
            schema = ('http://schema.org/%s' %(rel))
            year = int(triple[1])
            year = Literal(year, datatype=XSD.gYear)
            try:
                parse(schema, rule='IRI')
            except:
                n+=1
                print(n,'bad -',triple)
                continue
            g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), year ))
        else:
            obj = triple[1].replace("'", '')
            obj = obj.replace('"', '')
            obj = obj.replace('`', '')
            obj = obj.replace('.', '')
            link = 'http://yago-knowledge.org/resource/%s' %(obj)
            if triple[0] == 'type':
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(link)
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), RDF.type, rdflib.URIRef(link)) )
            else:
                rel = triple[0].replace("'", '')
                rel = rel.replace('"', '')
                rel = rel.replace('`', '')
                rel = rel.replace('.', '')
                schema = ('http://schema.org/%s' %(rel))
                try:
                    parse(link, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                try:
                    parse(schema, rule='IRI')
                except:
                    n+=1
                    print(n,'bad -',triple)
                    continue
                g.add( (rdflib.URIRef(s), rdflib.URIRef(schema), rdflib.URIRef(link)) )
    k+=1

1 bad - ['##bro', 'Pictures']
2 bad - ['##sIn', 'population']
3 bad - ['##sIn', 'Fyre_Festival']
4 bad - ['outOf', '48.6_%']
5 bad - ['received', '48.6_%']
6 bad - ['wasUp', '48.6_%']
7 bad - ['outOf', '55.9_%']
8 bad - ['##cSalt', 'salt']
9 bad - ['##nBesides', 'powerhouse']
10 bad - ['##nWith', 'Timbuktu']
11 bad - ['##nBy', 'Assimi_Goïta']
12 bad - ['##sOf', 'subgenre']
13 bad - ['##sOf', 'punk_rock']
14 bad - ['##sIn', 'late_1970s']
15 bad - ['##sWith', 'country']
16 bad - ['##niAs', 'Daily_Telegraph']
17 bad - ['##niAs', 'hardman']
18 bad - ['##sBeing', 'generations']
19 bad - ['##eIn', 'science_fiction']
20 bad - ['populationOf', '42.6_%']
21 bad - ['with', '42.6_%']
22 bad - ['identifying', '42.6_%']
23 bad - ['##edBy', '1992_Revisionist_Western_film']
24 bad - ['##edBy', 'Clint_Eastwood']
25 bad - ['##edBy', '1992_American_Revisionist_Western_film']
26 bad - ['##edWith', 'praise']
27 bad - ['##edBy', 'preservation']
28 bad - ['##edBy', 'Library']
29 bad - ['##nessOf', 'preserva

In [43]:
total = 0
for triples in data['StanfordFinalProTrips']:
    if len(triples) == 0:
        continue 
    else:
        for triple in triples:
            total += 1
total

348686

In [44]:
n,total, n/total

(6442, 348686, 0.018475074995841532)

6442/348686 triples were removed for not fitting the proper format, thats 1.84%

In [45]:
g.serialize(destination='Stanfordfinal.ttl', format='turtle')

<Graph identifier=Nd0a7b7fa66414d2cb1ea804276890bdb (<class 'rdflib.graph.Graph'>)>